## Preprocessing Pipeline

Preprocessing of raw images before landmark extraction for analysis of cxcr7 distribution. 

This includes **masking** of the entire neuromast and **membrane subtraction** to remove the membrane-localized signal in the cxcr7 channel based on the Lyn channel. Note that the masking **assumes the top 50 pixels (`stack[:,:50,:]`) to be background** and calculates the threshold based on that.

**Note:** 8-bit conversion is done before this, using the Fiji macro `8bit_macro.ijm`. A fixed conversion range is used that is kept the same across a given experiment. Minima are always 0 or 10000 (depending on airyscan settings), maxima are adjusted based on intensity range; the values are logged in `data\metadata.xlsx`.

### Preparations

In [ ]:
# External imports
from __future__ import division
import os, sys
import numpy as np
import scipy.ndimage as ndi
import matplotlib.pyplot as plt
from tifffile import imsave
%matplotlib inline

# Internal imports
import quant.preprocessing as prep

### Single Run for Testing

#### Masking

In [ ]:
# Specify input data
fpath = r'data_ex'
fname = r'example_stack_wt_8bit.tif'

In [ ]:
# Load data
stack = prep.load_data(fname, fpath)

In [ ]:
# Quick check of histogram of background region (should be only low values)
plt.figure(figsize=(4,2))
plt.hist(stack[:,:50,:].flatten(), bins=256)
plt.xlim([0,20])
plt.show()

In [ ]:
# Create neuromast mask
mask = prep.create_nm_mask(stack[0,...], show=True)

In [ ]:
# Apply mask
masked_stack = np.copy(stack)
masked_stack[0,~mask] = 0
masked_stack[1,~mask] = 0

In [ ]:
# Save output
imsave(os.path.join(fpath, fname[:-4]+'_masked.tif'), np.rollaxis(masked_stack,1), bigtiff=True)

#### Membrane Subtraction

In [ ]:
# Mean-normalize and subtract membrane channel from cxcr7 channel
subtracted = prep.subtract_mem(masked_stack[1,...], masked_stack[0,...])

In [ ]:
# Save output
imsave(os.path.join(fpath, fname[:-4]+'_masked_memsub.tif'), subtracted, bigtiff=True)

### Batch Run

In [ ]:
# Specify input parameters
fname_end = r'_8bit.tif'
fpath = r'data_ex'

# Number of process to use
processes = 10

# Get filenames (ready for multiprocessing)
fnames = [(fname, fpath) for fname in os.listdir(fpath) if fname.endswith(fname_end)]

# Run multiprocessed
prep.run_multiprocessed(fnames, processes)

# Report
print "### ALL DONE! ###"